In [ ]:
%matplotlib inline
from astropy.table import Table, Column
from matplotlib import pyplot as plt
from skimage import io
import numpy as np
from IPython import display

In [ ]:
results = Table.read('./catalogs/blank_table.fits')

In [ ]:
# add flag for edge of field
results.add_column(Column(data=-np.ones(len(results), dtype=int), name='EDGE'))

In [ ]:
results

# Now we are gonna visually inspect the blank fields

This is gonna let us identify which is the "primary" source

In [ ]:
# use the pandas iterrows because the astropy one sucks
results_df = results.to_pandas()

In [ ]:
# make a catalog mask

fig = plt.figure(1, figsize=(10, 10))
ax = fig.add_subplot()

plt.ion()

name_old= ''
for index, row in results_df.iterrows():

    if row.XRT_EXP < 1000:
        continue
    
    try:
        name = row.NAME.decode().replace(' ', '_')

        if name == name_old:
            pass
        else:
            display.clear_output(wait=True)
            jpgfile = f'../data/{name}/{name}_XRT_vtp.png'
            try:
                jpg_array = io.imread(jpgfile)
            except FileNotFoundError:
                name_old = name
                continue
                
            name_old = name
            
            ax.imshow(jpg_array, origin='upper', interpolation='bicubic')
            ax.set_title(f'{name}')

            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.yaxis.set_ticks([])
            ax.xaxis.set_ticks([])

            plt.show()
            display.display(fig)
            
    except NameError:
        display.clear_output(wait=True)
        jpgfile = f'../data/{name}/{name}_XRT_vtp.png'
        try:
            jpg_array = io.imread(jpgfile)
        except FileNotFoundError:
            name_old = name
            continue
        name_old = name
        
        ax.imshow(jpg_array, origin='upper', interpolation='bicubic')
        ax.set_title(f'{name}')

        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.yaxis.set_ticks([])
        ax.xaxis.set_ticks([])

        plt.show()
        display.display(fig)

    x = input(f"Is the PSZ location on the edge? [n]y.")
    
    if x == 'y':
        results_df.loc[index, 'EDGE'] = 1
    else:
        results_df.loc[index, 'EDGE'] = 0

# Save the results!

So we don't have to look at all of them again. 

Uncomment the next cell if you are gonna redo the inspections!

In [ ]:
results_df

In [ ]:
results_df = results_df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
results = Table.from_pandas(results_df)
results.filled(-1.0).write('./catalogs/blank_table_inspected.fits', overwrite=True)